In [ ]:
import os
import torch
from jiwer import wer, cer
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# Define the device based on CUDA availability
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Set torch data type based on CUDA availability
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Specify the model ID and the directory containing audio file
model_id = "openai/whisper-large-v3"
audio_directory = '/content/drive/MyDrive/common_voice_test'  # Adjust the path as needed

def load_model_and_processor():
    # Load the pre-trained model and processor for transcription.
    # Load the model and processor for speech-to-text conversion
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    return model, processor

def get_transcription(audio_file, model, processor):
    # Retrieve transcription for a given audio file using the provided model and processor.
    # Get the full path to the audio file
    audio_path = os.path.join(audio_directory, audio_file)

    # Use pipeline for automatic speech recognition
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        torch_dtype=torch_dtype,
        device=device,
    )
    result = pipe(audio_path)
    return result["text"]

def load_reference_transcriptions():
    # Load reference transcriptions from the provided file.
    # Read and process the reference transcriptions from the 'trans.txt' file
    transcriptions = {}
    with open(os.path.join(audio_directory, 'trans.txt'), 'r', encoding='utf-8') as file:
        trans_txt_content = file.readlines()

    for line in trans_txt_content:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            transcriptions[parts[0]] = parts[1]
    return transcriptions

def evaluate_transcriptions():
    # Evaluate transcriptions and calculate Word Error Rate (WER) and Character Error Rate (CER).
    # Get a list of audio files from the specified directory
    audio_files = [
        file for file in os.listdir(audio_directory) if file.endswith(".wav")
    ]

    # Load reference transcriptions from 'trans.txt'
    transcriptions = load_reference_transcriptions()
    total_wer = total_cer = total_sentences = 0

    # Iterate through audio files for transcription evaluation
    for audio_name in audio_files:
        # Obtain the transcription for the current audio file
        transcription = get_transcription(audio_name, model, processor)
        reference = transcriptions.get(audio_name)

        if reference is not None:
            print(f"Audio File: {audio_name}")
            print("Transcribed text:", transcription)
            print("Reference text:", reference)

            # Calculate WER and CER for the reference and obtained transcriptions
            curr_wer = wer(reference, transcription)
            curr_cer = cer(reference, transcription)

            # Accumulate total WER, CER, and sentences for average calculation
            total_wer += curr_wer
            total_cer += curr_cer
            total_sentences += 1

            # Display information for each audio file
            print("WER:", curr_wer)
            print("CER:", curr_cer)
            print("--------------------------------------")

    # Calculate average WER and CER for the evaluated transcriptions
    avg_wer = total_wer / total_sentences
    avg_cer = total_cer / total_sentences

    print(f"Average WER: {avg_wer}")
    print(f"Average CER: {avg_cer}")

if __name__ == "__main__":
    # Load the model and processor, then evaluate transcriptions
    model, processor = load_model_and_processor()
    evaluate_transcriptions()

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

Audio File: common_voice_mr_30956171.wav
Transcribed text:  ताटीच अभंगा द्वारे ग्यानेश्वरानी विनवनी ग्यानेश्वरान कडून ग्यानेश्वरी निर्मितीची मुक्ता बाईने चांग देवाना प
Reference text: ताटीच्या अभंगाद्वारे ज्ञानेश्वरांना विनवणी ज्ञानेश्वरांकडून ज्ञानेश्वरीची निर्मिती मुक्ताबाईने चांगदेवाना पासष्टी चा अर्थ सांगितला
WER: 1.0769230769230769
CER: 0.3384615384615385
--------------------------------------
Audio File: common_voice_mr_31120374.wav
Transcribed text:  ते थिल सर्वर अस्त्यांची व्यवस्था पाहर नेहे त्यांचेच काम।
Reference text: तेथील सर्व रस्त्यांची व्यवस्था पाहणे हे त्यांचेच काम
WER: 0.875
CER: 0.17307692307692307
--------------------------------------
Audio File: common_voice_mr_31208846.wav
Transcribed text:  यती पद्याचालित मनताना जियाक्षर उच्छाडल्या अंतर किंचित थामबावे लगते त्या थामनाला यती मनतात।
Reference text: यती पद्य चालीत म्हणताना जे अक्षर उच्चारल्यानंतर किंचित थांबावे लागते त्या थांबण्याला यती म्हणतात
WER: 0.7142857142857143
CER: 0.23958333333333334
------------------------

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Audio File: common_voice_mr_31147740.wav
Transcribed text:  पुर्वार्ध ब्रेटिशान नी अपल्या राज्य कार्भारा साथी भारताची विभागनी वेग-वेगला प्रांतात केली होती, परंतु ती भाषे प
Reference text: पूर्वार्ध ब्रिटिशांनी आपल्या राज्यकारभारासाठी भारताची विभागणी वेगवेगळ्या प्रांतात केली होती परंतु ती भाषेप्रमाणे नव्हती
WER: 0.8571428571428571
CER: 0.23529411764705882
--------------------------------------
Audio File: common_voice_mr_31069672.wav
Transcribed text:  अलीकर चा कालात मुंबाईत या पक्षाने आग्मन केले होते हैं।
Reference text: अलीकडच्या काळात मुंबईत या पक्षाने आगमन केले होते
WER: 0.75
CER: 0.25
--------------------------------------
Audio File: common_voice_mr_31138447.wav
Transcribed text:  जय प्रकाश नारायन यानी स्थपन केला समाजवादी पक्षव या त्या तुकड़ें परीकीच एक पक्षव।
Reference text: जयप्रकाश नारायण यांनी स्थापन केलेला समाजवादी पक्ष हा त्या तुकड्यांपैकीच एक पक्ष
WER: 0.9166666666666666
CER: 0.20253164556962025
--------------------------------------
Audio File: common_voice_mr_31208838.wav

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Audio File: common_voice_mr_31120247.wav
Transcribed text:  હે રાજ્ય શૃષ્ટી સોંદર્યાને વવિવીદો લોકો સંસ્ક�
Reference text: हे राज्य सृष्टिसौंदर्याने व विविध लोकसंस्कृतींनी नटले आहे
WER: 1.0
CER: 0.9298245614035088
--------------------------------------
Audio File: common_voice_mr_31195666.wav
Transcribed text:  अकोला जिल्यातिल तालुके अकोला, बालापूर, पातूर, बारशी टाकली, मुर्तिजापूर, अकोट वतेदलारा
Reference text: अकोला जिल्ह्यातील तालुके अकोला बाळापूर पातूर बार्शीटाकळी मुर्तिजापूर अकोट व तेल्हारा
WER: 0.8181818181818182
CER: 0.19047619047619047
--------------------------------------
Audio File: common_voice_mr_31195437.wav
Transcribed text:  विश्म प्रतिद्ञ्या देववरत हा हस्तिनापुरासा राजा शंतनु वगंगा यांचा पुत्र होता।
Reference text: भीष्मप्रतिज्ञा देवव्रत हा हस्तिनापुराचा राजा शंतनू व गंगा यांचा पुत्र होता
WER: 0.7272727272727273
CER: 0.16216216216216217
--------------------------------------
Audio File: common_voice_mr_31195433.wav
Transcribed text:  मरधे करोत तर नव कवीन चा दुसरा पेडीति